In [1]:
import os
from PIL import Image, ImageFilter
import numpy as np

In [ ]:
input_dir = 'archive/T91'
output_dir_lr = 'sub_images_lr'
output_dir_hr = 'sub_images_hr'
sub_image_size = 32
stride = 14
scale = 2 #in this implementation we are taking the upscaling factor of 2 only
blur_radius = 1.0

if not os.path.exists(output_dir_lr):
    os.makedirs(output_dir_lr)

if not os.path.exists(output_dir_hr):
    os.makedirs(output_dir_hr)

count = 0 # just to keep track of the number of patches
for filename in os.listdir(input_dir):
    filepath = os.path.join(input_dir, filename)

    # First we will do it for greyscale images and then will extend it to color images
    # Convert to YCrCb
    img = Image.open(filepath).convert('YCbCr')
    img = img.split()[0] # get the Y channel

    h, w = img.size

    for i in range(0, h - sub_image_size + 1, stride):
        for j in range(0, w - sub_image_size + 1, stride):
            # get the high resolution sub-image
            hr_sub_image = img.crop((j, i, j + sub_image_size, i + sub_image_size))

            #add gaussian noise to the patch
            hr_patch_image = hr_sub_image.filter(ImageFilter.GaussianBlur(blur_radius))

            # Downsample and upsample the patch 
            # subsample the image by scale
            lr_patch_image = hr_patch_image.resize((sub_image_size//scale, sub_image_size//scale), Image.BICUBIC)
            # upsample the image by scale
            lr_patch_image = lr_patch_image.resize((sub_image_size, sub_image_size), Image.BICUBIC)

            # Save the sub-images
            lr_patch_image.save(os.path.join(output_dir_lr, filename.split('.')[0] + '_' + str(i) + '_' + str(j) + '.png'))
            hr_patch_image.save(os.path.join(output_dir_hr, filename.split('.')[0] + '_' + str(i) + '_' + str(j) + '.png'))

            count += 1

By making sub image in a systematic way we were able to generate 22227 images of size 32X32 with the stride of 14X14

In [3]:
print(count)

22227
